In [20]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
import re
from collections import defaultdict
import zipfile
import pyarrow as pa
import pyarrow.parquet as pq
from gsscoder_python import recode_gss
import urllib.request
import pyreadr

In [ ]:
# Step 1
def ons_scraper(event=None, context=None):
    print("Starting ONS data scraping...")

    base_url = "https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/internalmigrationinenglandandwales/"
    print(f"Fetching base page: {base_url}")
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    print("Parsing download links...")
    links = [a['href'] for a in soup.find_all('a', href=True) if 'detailedinternalmigrationestimates' in a['href']]
    print(f"Found {len(links)} relevant links.")

    # Group links by year (assuming year is in the URL or filename)
    year_to_links = defaultdict(list)
    year_pattern = re.compile(r"(20\d{2})")  # Matches years like 2017, 2022, etc.

    for link in links:
        full_url = f"https://www.ons.gov.uk{link}"
        match = year_pattern.search(full_url)
        if match:
            year = match.group(1)
            year_to_links[year].append(full_url)

    print(f"Years detected: {list(year_to_links.keys())}")

    # Create test_data directory if it doesn't exist
    os.makedirs("test_data/1_raw", exist_ok=True)
    download_links = []

    for year, year_links in sorted(year_to_links.items()):
        sample_url = year_links[0]  # Pick the first link as sample for the year
        filename = sample_url.split("/")[-1]
        local_path = f"test_data/1_raw/{filename}"

        print(f"Downloading sample for {year}: {sample_url}")
        r = requests.get(sample_url)
        with open(local_path, "wb") as f:
            f.write(r.content)
        print(f"Saved to local path: {local_path}")

        download_links.append(local_path)

    print("Sample downloads saved locally to 'test_data/1_raw/'.")

    return {
        "statusCode": 200,
        "download_links": download_links
    }


In [ ]:
#Step 2
def local_clean_data(input_dir="test_data/1_raw", output_dir="test_data/2_clean_data"):
    print("Starting local clean_data process...")

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # List Excel files in input directory
    files = [f for f in os.listdir(input_dir) if f.endswith(".xls") or f.endswith(".xlsx")]
    if not files:
        print("No Excel files found to process.")
        return {"status": "no files"}

    print(f"Found {len(files)} files to process.")

    for filename in files:
        input_path = os.path.join(input_dir, filename)
        print(f"\nReading Excel file {input_path}...")

        try:
            df = pd.read_excel(input_path, sheet_name=4)
        except Exception as e:
            print(f"❌ Failed to read {filename}: {e}")
            continue

        print("Cleaning data (dropping NA rows)...")
        df_clean = df.dropna()

        # Modify filename
        clean_filename = filename.replace('2021and2023', '2023').replace('.xlsx', '.csv').replace('.xls', '.csv')
        output_path = os.path.join(output_dir, f"clean_{clean_filename}")

        print(f"Saving cleaned file to {output_path}...")
        try:
            df_clean.to_csv(output_path, index=False)
        except Exception as e:
            print(f"❌ Failed to write {output_path}: {e}")
            continue

        print(f"✅ Finished processing {filename}")

    print("All files processed.")
    return {"status": "done", "files_processed": len(files)}

In [ ]:
#Step 3

def local_combine_cleaned_data(input_dir="test_data/2_clean_data", output_path="test_data/combined/3_clean_data_combined.parquet"):
    print("Starting local combine_cleaned_data process...")

    if not os.path.isdir(input_dir):
        print(f"Input directory does not exist: {input_dir}")
        return {"status": "error", "message": f"{input_dir} not found"}

    # Ensure output directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    all_files = [f for f in os.listdir(input_dir) if f.endswith(".csv")]
    if not all_files:
        print("No CSV files found.")
        return {"status": "no files"}

    print(f"Found {len(all_files)} files to combine.")

    combined_df = pd.DataFrame()

    for file in all_files:
        file_path = os.path.join(input_dir, file)
        print(f"Reading file: {file_path}")
        try:
            df = pd.read_csv(file_path)
            df.columns = df.columns.str.lower()  # column headers to lowercase
            combined_df = pd.concat([combined_df, df], ignore_index=True)
        except Exception as e:
            print(f"❌ Failed to read {file}: {e}")

    # Save to Parquet
    print(f"Saving combined data to {output_path}...")
    combined_df.to_parquet(output_path, index=False)
    print("✅ Combined file successfully written.")

    return {
        "status": "done",
        "files_combined": len(all_files),
        "output_file": output_path
    }


In [ ]:
#Step 4

# ----------------------------- Configuration -----------------------------
# Local equivalents of your S3 paths
LOCAL_PARQUET_ZIP_PATH = "test_data/old_series_data/origin_destination_2002_to_2020.parquet.zip"
LOCAL_NEW_SERIES_PARQ = "test_data/combined/3_clean_data_combined.parquet"
LOCAL_OUTPUT_DIR = "test_data/new_geog_combined/output_parquet"

START_YR_NEW_SERIES = 2012
GSS_OLD_YEAR = 2021
GSS_NEW_YEAR = 2023


def combine_series_locally():
    print("🚀 Starting local combine_series...")

    # 1. Unzip and load the old series Parquet
    print("📥 Reading ZIP from local file...")
    with zipfile.ZipFile(LOCAL_PARQUET_ZIP_PATH, 'r') as zip_ref:
        parquet_name = zip_ref.namelist()[0]
        print(f"📦 Unzipping and reading {parquet_name}")
        with zip_ref.open(parquet_name) as f:
            df_old = pd.read_parquet(f)

    df_old = df_old[df_old['year'] < START_YR_NEW_SERIES]
    print(f"✅ Loaded old series shape: {df_old.shape}")

    # 2. Recode gss_in
    df_in_gss = df_old[df_old['gss_in'].str.contains("E0|W0")]
    df_not_in_gss = df_old[~df_old['gss_in'].str.contains("E0|W0")]

    print(f"🔄 Recoding gss_in from {GSS_OLD_YEAR} to {GSS_NEW_YEAR}...")
    df_in_gss_recoded = recode_gss(
        df_in=df_in_gss,
        col_code='gss_in',
        col_data='value',
        fun='sum',
        recode_from_year=GSS_OLD_YEAR,
        recode_to_year=GSS_NEW_YEAR,
    )

    df_old_recoded_in = pd.concat([df_in_gss_recoded, df_not_in_gss], ignore_index=True)

    # 3. Recode gss_out
    df_out_gss = df_old_recoded_in[df_old_recoded_in['gss_out'].str.contains("E0|W0")]
    df_not_out_gss = df_old_recoded_in[~df_old_recoded_in['gss_out'].str.contains("E0|W0")]

    print(f"🔄 Recoding gss_out from {GSS_OLD_YEAR} to {GSS_NEW_YEAR}...")
    df_out_gss_recoded = recode_gss(
        df_in=df_out_gss,
        col_code='gss_out',
        col_data='value',
        fun='sum',
        recode_from_year=GSS_OLD_YEAR,
        recode_to_year=GSS_NEW_YEAR,
    )

    df_old_recoded = pd.concat([df_out_gss_recoded, df_not_out_gss], ignore_index=True)
    print(f"✅ Recoded old series shape: {df_old_recoded.shape}")

    # 4. Load new series CSV
    print(f"📥 Reading new series CSV from {LOCAL_NEW_SERIES_PARQ}...")
    new_df = pd.read_parquet(LOCAL_NEW_SERIES_PARQ)
    print(f"✅ Loaded new series shape: {new_df.shape}")

    # 5. Combine and filter
    df_combined = pd.concat([df_old_recoded, new_df], ignore_index=True)
    df_combined = df_combined[df_combined['gss_in'] != df_combined['gss_out']]
    print(f"🧩 Combined series shape after filtering: {df_combined.shape}")

    # 6. Partition and write Parquet by year
    grouped = df_combined.groupby("year")
    for year, group in grouped:
        table = pa.Table.from_pandas(group)
        output_path = os.path.join(LOCAL_OUTPUT_DIR, f"year={year}")
        os.makedirs(output_path, exist_ok=True)
        pq.write_to_dataset(table, root_path=output_path, partition_cols=["year"])
        print(f"💾 Written Parquet partition for year {year} to {output_path}")

    print("✅ All done.")
    return {"status": "done", "years_written": grouped.ngroups}

In [24]:
#Step 6 

# ------------------ Configuration ------------------ #
POPULATION_PATH = "data/processed/population_coc.rds"
URL_POPULATION = (
    "https://data.london.gov.uk/download/modelled-population-backseries/"
    "2b07a39b-ba63-403a-a3fc-5456518ca785/full_modelled_estimates_series_EW%282023_geog%29.rds"
)

# ------------------ Ensure Directory Exists ------------------ #
output_dir = os.path.dirname(POPULATION_PATH)
os.makedirs(output_dir, exist_ok=True)

# ------------------ Download if Missing ------------------ #
if not os.path.isfile(POPULATION_PATH):
    print("⬇️  Downloading modelled population estimates...")
    try:
        urllib.request.urlretrieve(URL_POPULATION, POPULATION_PATH)
        print(f"✅ Saved to: {POPULATION_PATH}")
    except Exception as e:
        print(f"❌ Download failed: {e}")
else:
    print(f"✅ File already exists at: {POPULATION_PATH}")



✅ File already exists at: data/processed/population_coc.rds


In [13]:
# Step 1 Run locally 
if __name__ == "__main__":
    result = ons_scraper()
    print(result)

Starting ONS data scraping...
Fetching base page: https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/internalmigrationinenglandandwales/
Parsing download links...
Found 11 relevant links.
Years detected: ['2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012']
Saved to local path: test_data/raw/detailedestimates2012on2021and2023las.xlsx
Saved to local path: test_data/raw/detailedestimates2013on2021and2023las.xlsx
Saved to local path: test_data/raw/detailedestimates2014on2021and2023las.xlsx
Saved to local path: test_data/raw/detailedestimates2015on2021and2023las.xlsx
Saved to local path: test_data/raw/detailedestimates2016on2021and2023las.xlsx
Saved to local path: test_data/raw/detailedestimates2017on2021and2023las.xlsx
Saved to local path: test_data/raw/detailedestimates2018on2021and2023las.xlsx
Saved to local path: test_data/raw/detailedestimates2019on2021and2023las.xlsx
Saved to local path: test_data

In [19]:
# Step 2 Run locally
if __name__ == "__main__":
    result = local_clean_data()
    print(result)

Starting local clean_data process...
Found 11 files to process.

Reading Excel file test_data/raw/detailedestimates2013on2021and2023las.xlsx...
Cleaning data (dropping NA rows)...
Saving cleaned file to test_data/clean_data/clean_detailedestimates2013on2023las.csv...
✅ Finished processing detailedestimates2013on2021and2023las.xlsx

Reading Excel file test_data/raw/detailedestimates2016on2021and2023las.xlsx...
Cleaning data (dropping NA rows)...
Saving cleaned file to test_data/clean_data/clean_detailedestimates2016on2023las.csv...
✅ Finished processing detailedestimates2016on2021and2023las.xlsx

Reading Excel file test_data/raw/detailedestimates2021on2021and2023las.xlsx...
Cleaning data (dropping NA rows)...
Saving cleaned file to test_data/clean_data/clean_detailedestimates2021on2023las.csv...
✅ Finished processing detailedestimates2021on2021and2023las.xlsx

Reading Excel file test_data/raw/detailedestimates2019on2021and2023las.xlsx...
Cleaning data (dropping NA rows)...
Saving cleane

In [34]:
# Step 3 Run locally
if __name__ == "__main__":
    result = local_combine_cleaned_data()
    print(result)

Starting local combine_cleaned_data process...
Found 11 files to combine.
Reading file: test_data/clean_data/clean_detailedestimates2018on2023las.csv
Reading file: test_data/clean_data/clean_detailedestimates2019on2023las.csv
Reading file: test_data/clean_data/clean_detailedestimates2013on2023las.csv
Reading file: test_data/clean_data/clean_detailedestimates2014on2023las.csv
Reading file: test_data/clean_data/clean_detailedestimates2022on2023las.csv
Reading file: test_data/clean_data/clean_detailedestimates2015on2023las.csv
Reading file: test_data/clean_data/clean_detailedestimates2012on2023las.csv
Reading file: test_data/clean_data/clean_detailedestimates2017on2023las.csv
Reading file: test_data/clean_data/clean_detailedestimates2021on2023las.csv
Reading file: test_data/clean_data/clean_detailedestimates2016on2023las.csv
Reading file: test_data/clean_data/clean_detailedestimates2020on2023las.csv
Saving combined data to test_data/combined/cleaned_data_combined.parquet...
✅ Combined fil

In [12]:
# step 4 Run locally
if __name__ == "__main__":
    result = combine_series_locally()
    print(result)


🚀 Starting local combine_series...
📥 Reading ZIP from local file...
📦 Unzipping and reading origin_destination_2002_to_2020.parquet
✅ Loaded old series shape: (19705748, 6)
🔄 Recoding gss_in from 2021 to 2023...
🔄 Recoding gss_out from 2021 to 2023...
✅ Recoded old series shape: (19705748, 6)
📥 Reading new series CSV from test_data/combined/cleaned_data_combined.parquet...
✅ Loaded new series shape: (1806380, 116)
🧩 Combined series shape after filtering: (21484481, 120)
💾 Written Parquet partition for year 2002.0 to test_data/new_geog_combined/output_parquet/year=2002.0
💾 Written Parquet partition for year 2003.0 to test_data/new_geog_combined/output_parquet/year=2003.0
💾 Written Parquet partition for year 2004.0 to test_data/new_geog_combined/output_parquet/year=2004.0
💾 Written Parquet partition for year 2005.0 to test_data/new_geog_combined/output_parquet/year=2005.0
💾 Written Parquet partition for year 2006.0 to test_data/new_geog_combined/output_parquet/year=2006.0
💾 Written Parqu

--------------------------------------------------------------------------------------

In [3]:
#test step 4

In [ ]:
LOCAL_PARQUET_ZIP_PATH = "test_data/old_series_data/origin_destination_2002_to_2020.parquet.zip"
LOCAL_NEW_SERIES_PARQ = "test_data/combined/3_clean_data_combined.parquet"
LOCAL_OUTPUT_DIR = "test_data/new_geog_combined/output_parquet"

START_YR_NEW_SERIES = 2012
GSS_OLD_YEAR = 2021
GSS_NEW_YEAR = 2023

In [14]:
print("🚀 Starting local combine_series...")
# 1. Unzip and load the old series Parquet
print("📥 Reading ZIP from local file...")
with zipfile.ZipFile(LOCAL_PARQUET_ZIP_PATH, 'r') as zip_ref:
    parquet_name = zip_ref.namelist()[0]
    print(f"📦 Unzipping and reading {parquet_name}")
    with zip_ref.open(parquet_name) as f:
        df_old = pd.read_parquet(f)

🚀 Starting local combine_series...
📥 Reading ZIP from local file...
📦 Unzipping and reading origin_destination_2002_to_2020.parquet


In [6]:
df_old

,gss_out,gss_in,year,sex,age,value
0,E06000001,E06000002,2002.0,female,7.0,1.1805
1,E06000001,E06000002,2002.0,female,8.0,1.1805
2,E06000001,E06000002,2002.0,female,9.0,1.1805
3,E06000001,E06000002,2002.0,female,11.0,2.3610
4,E06000001,E06000002,2002.0,female,12.0,1.1805
...,...,...,...,...,...,...
31196466,W06000024,W06000023,2020.0,male,34.0,1.1089
31196467,W06000024,W06000023,2020.0,male,35.0,1.1565
31196468,W06000024,W06000023,2020.0,male,43.0,1.1501
31196469,W06000024,W06000023,2020.0,male,66.0,1.1905


In [7]:
df_old = df_old[df_old['year'] < START_YR_NEW_SERIES]
print(f"✅ Loaded old series shape: {df_old.shape}")

✅ Loaded old series shape: (19705748, 6)


In [8]:
df_old

,gss_out,gss_in,year,sex,age,value
0,E06000001,E06000002,2002.0,female,7.0,1.1805
1,E06000001,E06000002,2002.0,female,8.0,1.1805
2,E06000001,E06000002,2002.0,female,9.0,1.1805
3,E06000001,E06000002,2002.0,female,11.0,2.3610
4,E06000001,E06000002,2002.0,female,12.0,1.1805
...,...,...,...,...,...,...
29937902,W06000024,W06000023,2011.0,male,34.0,1.1264
29937903,W06000024,W06000023,2011.0,male,38.0,1.1264
29937904,W06000024,W06000023,2011.0,male,43.0,1.1264
29937905,W06000024,W06000023,2011.0,male,52.0,1.1264


In [16]:
# 2. Recode gss_in
df_in_gss = df_old[df_old['gss_in'].str.contains("E0|W0")]
df_not_in_gss = df_old[~df_old['gss_in'].str.contains("E0|W0")]

print(f"🔄 Recoding gss_in from {GSS_OLD_YEAR} to {GSS_NEW_YEAR}...")
df_in_gss_recoded = recode_gss(
    df_in=df_in_gss,
    col_code='gss_in',
    col_data='value',
    fun='sum',
    recode_from_year=GSS_OLD_YEAR,
    recode_to_year=GSS_NEW_YEAR,
)

df_old_recoded_in = pd.concat([df_in_gss_recoded, df_not_in_gss], ignore_index=True)

🔄 Recoding gss_in from 2021 to 2023...


In [17]:
df_in_gss_recoded 

,gss_out,gss_in,year,sex,age,value
0,E06000001,E06000002,2002.0,female,7.0,1.1805
1,E06000001,E06000002,2002.0,female,8.0,1.1805
2,E06000001,E06000002,2002.0,female,9.0,1.1805
3,E06000001,E06000002,2002.0,female,11.0,2.3610
4,E06000001,E06000002,2002.0,female,12.0,1.1805
...,...,...,...,...,...,...
30687889,W06000024,W06000023,2020.0,male,34.0,1.1089
30687890,W06000024,W06000023,2020.0,male,35.0,1.1565
30687891,W06000024,W06000023,2020.0,male,43.0,1.1501
30687892,W06000024,W06000023,2020.0,male,66.0,1.1905


In [18]:
df_in_gss

,gss_out,gss_in,year,sex,age,value
0,E06000001,E06000002,2002.0,female,7.0,1.1805
1,E06000001,E06000002,2002.0,female,8.0,1.1805
2,E06000001,E06000002,2002.0,female,9.0,1.1805
3,E06000001,E06000002,2002.0,female,11.0,2.3610
4,E06000001,E06000002,2002.0,female,12.0,1.1805
...,...,...,...,...,...,...
31196466,W06000024,W06000023,2020.0,male,34.0,1.1089
31196467,W06000024,W06000023,2020.0,male,35.0,1.1565
31196468,W06000024,W06000023,2020.0,male,43.0,1.1501
31196469,W06000024,W06000023,2020.0,male,66.0,1.1905


In [19]:
# Compare 'gss_in' columns in df_in_gss_recoded and df_in_gss
gss_in_recoded = set(df_in_gss_recoded['gss_in'].unique())
gss_in_original = set(df_in_gss['gss_in'].unique())

added = gss_in_recoded - gss_in_original
removed = gss_in_original - gss_in_recoded

print(f"Number of unique gss_in in original: {len(gss_in_original)}")
print(f"Number of unique gss_in in recoded: {len(gss_in_recoded)}")
print(f"Added codes: {added}")
print(f"Removed codes: {removed}")

Number of unique gss_in in original: 331
Number of unique gss_in in recoded: 331
Added codes: set()
Removed codes: set()


In [11]:
df_old_recoded_in

,gss_out,gss_in,year,sex,age,value
0,E06000001,E06000002,2002.0,female,7.0,1.1805
1,E06000001,E06000002,2002.0,female,8.0,1.1805
2,E06000001,E06000002,2002.0,female,9.0,1.1805
3,E06000001,E06000002,2002.0,female,11.0,2.3610
4,E06000001,E06000002,2002.0,female,12.0,1.1805
...,...,...,...,...,...,...
19705743,W06000024,S92000003,2011.0,male,37.0,1.3854
19705744,W06000024,S92000003,2011.0,male,40.0,1.3854
19705745,W06000024,S92000003,2011.0,male,44.0,1.3854
19705746,W06000024,S92000003,2011.0,male,47.0,1.3854


In [12]:
# 3. Recode gss_out
df_out_gss = df_old_recoded_in[df_old_recoded_in['gss_out'].str.contains("E0|W0")]
df_not_out_gss = df_old_recoded_in[~df_old_recoded_in['gss_out'].str.contains("E0|W0")]

print(f"🔄 Recoding gss_out from {GSS_OLD_YEAR} to {GSS_NEW_YEAR}...")
df_out_gss = recode_gss(
    df_in=df_out_gss,
    col_code='gss_out',
    col_data='value',
    fun='sum',
    recode_from_year=GSS_OLD_YEAR,
    recode_to_year=GSS_NEW_YEAR,
)

df_old_recoded = pd.concat([df_out_gss, df_not_out_gss], ignore_index=True)
print(f"✅ Recoded old series shape: {df_old_recoded.shape}")

🔄 Recoding gss_out from 2021 to 2023...
✅ Recoded old series shape: (19705748, 6)


In [13]:
df_old_recoded

,gss_out,gss_in,year,sex,age,value
0,E06000001,E06000002,2002.0,female,7.0,1.1805
1,E06000001,E06000002,2002.0,female,8.0,1.1805
2,E06000001,E06000002,2002.0,female,9.0,1.1805
3,E06000001,E06000002,2002.0,female,11.0,2.3610
4,E06000001,E06000002,2002.0,female,12.0,1.1805
...,...,...,...,...,...,...
19705743,S92000003,W06000024,2011.0,female,32.0,1.0695
19705744,S92000003,W06000024,2011.0,male,0.0,0.0716
19705745,S92000003,W06000024,2011.0,male,21.0,1.0695
19705746,S92000003,W06000024,2011.0,male,35.0,1.0695


In [35]:
# 4. Load new series CSV
print(f"📥 Reading new series CSV from {LOCAL_NEW_SERIES_PARQ}...")
new_df = pd.read_parquet(LOCAL_NEW_SERIES_PARQ)
print(f"✅ Loaded new series shape: {new_df.shape}")

# 5. Combine and filter
df_combined = pd.concat([df_old_recoded, new_df], ignore_index=True)
df_combined = df_combined[df_combined['gss_in'] != df_combined['gss_out']]
print(f"🧩 Combined series shape after filtering: {df_combined.shape}")

📥 Reading new series CSV from test_data/combined/cleaned_data_combined.parquet...
✅ Loaded new series shape: (1806380, 116)
🧩 Combined series shape after filtering: (21484481, 120)


In [36]:
new_df

,outla,inla,sex,year,age_0,age_1,age_2,age_3,age_4,age_5,...,age_102,age_103,age_104,age_105,age_106,age_107,age_108,age_109,age_110,age_111
0,E06000001,E06000002,F,2018,0.3216,1.4373,2.9503,0.0000,0.0000,2.9367,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
1,E06000001,E06000002,M,2018,0.0000,0.0000,1.4466,4.5061,0.0000,3.0329,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
2,E06000001,E06000003,F,2018,0.0000,0.0000,0.0000,1.4404,1.5002,1.4801,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
3,E06000001,E06000003,M,2018,0.0000,0.0000,1.4519,0.0000,1.5057,1.4899,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
4,E06000001,E06000004,F,2018,3.5056,8.7275,10.3012,11.7748,5.9903,2.9396,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1806375,W06000024,W06000021,M,2020,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN
1806376,W06000024,W06000022,F,2020,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN
1806377,W06000024,W06000022,M,2020,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN
1806378,W06000024,W06000023,F,2020,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN


In [38]:
new_df
#year range
print(f"Year range in combined data: {new_df['year'].min()} to {new_df['year'].max()}")

Year range in combined data: 2012 to 2022


In [ ]:
df_old_recoded
print(f"Year range in combined data: {df_old_recoded['year'].min()} to {df_old_recoded['year'].max()}")

Year range in combined data: 2002.0 to 2011.0


In [5]:
#read in /Users/user1/Documents/domestic_rates_preprocessing/test_data/clean_data/clean_detailedestimates2022on2023las.csv
csv_new_data = pd.read_csv("test_data/clean_data/clean_detailedestimates2022on2023las.csv")

In [6]:
csv_new_data

,outla,inla,Sex,Year,Age_0,Age_1,Age_2,Age_3,Age_4,Age_5,...,Age_100,Age_101,Age_102,Age_103,Age_104,Age_105,Age_106,Age_107,Age_108,Age_109
0,E06000001,E06000002,F,2022,0.0000,0.0000,1.4775,1.4006,1.4377,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,E06000001,E06000002,M,2022,0.0000,0.0000,1.4019,0.0000,0.0000,1.4481,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,E06000001,E06000003,F,2022,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,E06000001,E06000003,M,2022,0.0000,0.0000,0.0000,0.0000,1.4501,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,E06000001,E06000004,F,2022,0.8027,4.2071,1.4585,5.6678,7.3003,2.8094,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165034,W06000024,W06000021,M,2022,0.0000,0.0000,0.0000,1.1745,0.0000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
165035,W06000024,W06000022,F,2022,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
165036,W06000024,W06000022,M,2022,0.0000,0.0000,0.0000,1.1747,0.0000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
165037,W06000024,W06000023,F,2022,0.5646,1.2099,0.0000,0.0000,0.0000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
def convert_age_data(df):
    
    # Rename key columns
    df = df.rename(columns={"outla": "gss_out", "inla": "gss_in", "Sex": "sex", "Year": "year"})

    # Melt age columns
    long_df = df.melt(
        id_vars=["gss_out", "gss_in", "sex", "year"],
        var_name="age",
        value_name="value"
    )

    # Clean up age and sex
    long_df["age"] = long_df["age"].str.replace("age_", "", regex=False).astype(float)
    long_df["sex"] = long_df["sex"].map({"F": "female", "M": "male"})
    long_df["year"] = long_df["year"].astype(float)

    # Remove zero or missing values if desired
    long_df = long_df[long_df["value"] != 0]

    return long_df.reset_index(drop=True)


In [47]:
new_df

,outla,inla,sex,year,age_0,age_1,age_2,age_3,age_4,age_5,...,age_102,age_103,age_104,age_105,age_106,age_107,age_108,age_109,age_110,age_111
0,E06000001,E06000002,F,2018,0.3216,1.4373,2.9503,0.0000,0.0000,2.9367,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
1,E06000001,E06000002,M,2018,0.0000,0.0000,1.4466,4.5061,0.0000,3.0329,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
2,E06000001,E06000003,F,2018,0.0000,0.0000,0.0000,1.4404,1.5002,1.4801,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
3,E06000001,E06000003,M,2018,0.0000,0.0000,1.4519,0.0000,1.5057,1.4899,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
4,E06000001,E06000004,F,2018,3.5056,8.7275,10.3012,11.7748,5.9903,2.9396,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1806375,W06000024,W06000021,M,2020,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN
1806376,W06000024,W06000022,F,2020,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN
1806377,W06000024,W06000022,M,2020,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN
1806378,W06000024,W06000023,F,2020,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN


In [48]:
new_series_converted = convert_age_data(new_df)


: 

In [27]:
new_df 

,outla,inla,sex,Year,Age_0,Age_1,Age_2,Age_3,Age_4,Age_5,...,Age_103,Age_104,Age_105,Age_106,Age_107,Age_108,Age_109,Age_110,Sex,Age_111
0,E06000001,E06000002,F,2018,0.3216,1.4373,2.9503,0.0000,0.0000,2.9367,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,None,NaN
1,E06000001,E06000002,M,2018,0.0000,0.0000,1.4466,4.5061,0.0000,3.0329,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,None,NaN
2,E06000001,E06000003,F,2018,0.0000,0.0000,0.0000,1.4404,1.5002,1.4801,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,None,NaN
3,E06000001,E06000003,M,2018,0.0000,0.0000,1.4519,0.0000,1.5057,1.4899,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,None,NaN
4,E06000001,E06000004,F,2018,3.5056,8.7275,10.3012,11.7748,5.9903,2.9396,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1806375,W06000024,W06000021,M,2020,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,None,NaN
1806376,W06000024,W06000022,F,2020,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,None,NaN
1806377,W06000024,W06000022,M,2020,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,None,NaN
1806378,W06000024,W06000023,F,2020,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,None,NaN


In [23]:
df_combined 

,gss_out,gss_in,year,sex,age,value,outla,inla,Year,Age_0,...,Age_103,Age_104,Age_105,Age_106,Age_107,Age_108,Age_109,Age_110,Sex,Age_111
0,E06000001,E06000002,2002.0,female,7.0,1.1805,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E06000001,E06000002,2002.0,female,8.0,1.1805,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E06000001,E06000002,2002.0,female,9.0,1.1805,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E06000001,E06000002,2002.0,female,11.0,2.3610,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E06000001,E06000002,2002.0,female,12.0,1.1805,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21512123,NaN,NaN,NaN,M,NaN,NaN,W06000024,W06000021,2020.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,None,NaN
21512124,NaN,NaN,NaN,F,NaN,NaN,W06000024,W06000022,2020.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,None,NaN
21512125,NaN,NaN,NaN,M,NaN,NaN,W06000024,W06000022,2020.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,None,NaN
21512126,NaN,NaN,NaN,F,NaN,NaN,W06000024,W06000023,2020.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,None,NaN


In [13]:
# 2. Recode gss_in
df_in_gss = df_old[df_old['gss_in'].str.contains("E0|W0")].copy()
df_not_in_gss = df_old[~df_old['gss_in'].str.contains("E0|W0")]

# Keep track of original gss_in values
original_codes = df_in_gss[['gss_in']].drop_duplicates().copy()

print(f"🔄 Recoding gss_in from {GSS_OLD_YEAR} to {GSS_NEW_YEAR}...")
df_in_gss = recode_gss(
    df_in=df_in_gss,
    col_code='gss_in',
    col_data='value',
    fun='sum',
    recode_from_year=GSS_OLD_YEAR,
    recode_to_year=GSS_NEW_YEAR,
)

# Track changes by comparing original and recoded
new_codes = df_in_gss[['gss_in']].drop_duplicates()

# Detect changes
changed_codes = original_codes.merge(new_codes, on='gss_in', how='outer', indicator=True)
changed_codes = changed_codes[changed_codes['_merge'] != 'both']

print("📝 Changed gss_in codes:")
print(changed_codes['gss_in'].tolist())

# Merge back with untouched rows
df_old_recoded_in = pd.concat([df_in_gss, df_not_in_gss], ignore_index=True)

🔄 Recoding gss_in from 2021 to 2023...
📝 Changed gss_in codes:
[]


In [12]:
# 3. Recode gss_out
df_out_gss = df_old_recoded_in[df_old_recoded_in['gss_out'].str.contains("E0|W0")]
df_not_out_gss = df_old_recoded_in[~df_old_recoded_in['gss_out'].str.contains("E0|W0")]

print(f"🔄 Recoding gss_out from {GSS_OLD_YEAR} to {GSS_NEW_YEAR}...")
df_out_gss = recode_gss(
    df_in=df_out_gss,
    col_code='gss_out',
    col_data='value',
    fun='sum',
    recode_from_year=GSS_OLD_YEAR,
    recode_to_year=GSS_NEW_YEAR,
    )

🔄 Recoding gss_out from 2021 to 2023...


In [4]:
changes = pd.read_csv("/Users/user1/Documents/gsscoder_python/gsscoder_python/lookups/Changes.csv")

/var/folders/k7/sydv9mbx0kz7fx7373rx_2100000gp/T/ipykernel_79789/146933824.py:1: DtypeWarning: Columns (1,2,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  changes = pd.read_csv("/Users/user1/Documents/gsscoder_python/gsscoder_python/lookups/Changes.csv")


In [5]:
changes

,GEOGCD,GEOGNM,GEOGNMW,GEOGCD_P,GEOGNM_P,GEOGNMW_P,SI_ID,SI_TITLE,OPER_DATE,ENTITYCD,YEAR
0,S00023582,NaN,NaN,60QS002202,NaN,NaN,1111/1001,GSS re-coding strategy,01/01/2009,S00,2009.0
1,S00023583,NaN,NaN,60QS002203,NaN,NaN,1111/1001,GSS re-coding strategy,01/01/2009,S00,2009.0
2,S00023584,NaN,NaN,60QS002204,NaN,NaN,1111/1001,GSS re-coding strategy,01/01/2009,S00,2009.0
3,S00023585,NaN,NaN,60QS002205,NaN,NaN,1111/1001,GSS re-coding strategy,01/01/2009,S00,2009.0
4,S00023586,NaN,NaN,60QS002206,NaN,NaN,1111/1001,GSS re-coding strategy,01/01/2009,S00,2009.0
...,...,...,...,...,...,...,...,...,...,...,...
504968,W04000302,Cwmdu and District,Cwm-du a’r Cylch,W04000302,Llanfihangel Cwmdu with Bwlch and Cathedine,NaN,6003/2023,Name Change - Powys order 2006,01/01/2023,W04,2023.0
504969,W04000331,Mochdre and Penstrowed,Mochdre gyda Phenystrywaid,W04000331,Mochdre,NaN,6003/2023,Name Change - Powys order 2006,01/01/2023,W04,2023.0
504970,W18000003,Eryri,Eryri,W18000003,Snowdonia,Eryri,NaN,Name Change,17/04/2023,W18,2023.0
504971,E05014284,Hunmanby & Sherburn,NaN,E05014284,Hunmanby & Sherburn (part Ryedale),NaN,328/2022,The North Yorkshire (Structural Changes) Order...,01/04/2023,E05,2022.0


In [7]:
#select years 2023 nd 2021
changes_2021_2023 = changes[changes['YEAR'].isin([2023, 2021])]

In [8]:
changes_2021_2023 

,GEOGCD,GEOGNM,GEOGNMW,GEOGCD_P,GEOGNM_P,GEOGNMW_P,SI_ID,SI_TITLE,OPER_DATE,ENTITYCD,YEAR
445533,NaN,NaN,NaN,E10000021,Northamptonshire,NaN,156/2020,The Northamptonshire (Structural Changes) Orde...,01/04/2021,E10,2021.0
445542,E06000061,North Northamptonshire,NaN,E07000150,Corby,NaN,156/2020,The Northamptonshire (Structural Changes) Orde...,01/04/2021,E06,2021.0
445543,E06000061,North Northamptonshire,NaN,E07000152,East Northamptonshire,NaN,156/2020,The Northamptonshire (Structural Changes) Orde...,01/04/2021,E06,2021.0
445544,E06000061,North Northamptonshire,NaN,E07000153,Kettering,NaN,156/2020,The Northamptonshire (Structural Changes) Orde...,01/04/2021,E06,2021.0
445545,E06000061,North Northamptonshire,NaN,E07000156,Wellingborough,NaN,156/2020,The Northamptonshire (Structural Changes) Orde...,01/04/2021,E06,2021.0
...,...,...,...,...,...,...,...,...,...,...,...
504966,W04000254,Betws Cedewain,Betws Cedewain,W04000254,Bettws,NaN,6003/2023,Name Change - Powys order 2006,01/01/2023,W04,2023.0
504967,W04000296,Abermule with Llandyssil,Aber-miwl gyda Llandysul,W04000296,Llandyssil,NaN,6003/2023,Name Change - Powys order 2006,01/01/2023,W04,2023.0
504968,W04000302,Cwmdu and District,Cwm-du a’r Cylch,W04000302,Llanfihangel Cwmdu with Bwlch and Cathedine,NaN,6003/2023,Name Change - Powys order 2006,01/01/2023,W04,2023.0
504969,W04000331,Mochdre and Penstrowed,Mochdre gyda Phenystrywaid,W04000331,Mochdre,NaN,6003/2023,Name Change - Powys order 2006,01/01/2023,W04,2023.0
